In [17]:
from search_mod import *
from helpers_mod import *

In [18]:
psource(Problem)
psource(Node)
psource(expand)
psource(path_actions)
psource(path_states)
psource(PriorityQueue)
psource(breadth_first_search)
psource(depth_first_recursive_search)
psource(depth_limited_search)

psource (report)
psource (CountCalls)
psource (report_counts)

In [27]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """
                                # MODIFICAR GOAL PARA CAMBIAR EL OBJETIVO DEL 8 PUZZLE
    def __init__(self, initial, goal = (0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2  
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    def h(self, node): return 0
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')

In [20]:
class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

In [21]:
# Ejemplos de creación de instancias del problema del 8-puzzle
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))

# resolver una instancia particular con una estrategia de búsqueda concreta
# retorna la solución, como una instancia de Node
print(f'Resolver el problema {e1} con Búsqueda en Anchura')
sol = breadth_first_search(e1)
print ("Nodo solución: ", sol)

Resolver el problema EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8), (0, 1, 2, 3, 4, 5, 6, 7, 8)) con Búsqueda en Anchura
Nodo solución:  <(0, 1, 2, 3, 4, 5, 6, 7, 8)>


In [22]:
# INICIALIZACIÓN DEL PROBLEMA
EJERCICIO = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
print(f'Resolver el problema {EJERCICIO} con Búsqueda en Anchura')
solucion = breadth_first_search(EJERCICIO)
print ("Nodo solución: ", solucion)


# COSTE HASTA LLEGAR A LA SOLUCIÓN (NÚMERO DE TRANSICIONES, EN EL 8 PUZZLE)
print (f'Coste del camino: {sol.path_cost:d}')

# ESTADO DE LA SOLUCIÓN (EN TUPLA)
print (f'Estado solución: {sol.state:}')

# ESTADO DE LA SOLUCIÓN (EN TABLERO)
print (f'Estado solución (tablero):')
print (board8(sol.state))

# LISTA CON EL CONJUNTO DE ACCIONES
print (f'Lista de acciones: {path_actions(sol)}')

# NÚMERO DE ACCIONES REALIZADAS (LONGITUD DE LA LISTA ANTERIOR)
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')

# LISTA CON EL CONJUNTO DE ESTADOS (DESDE EL ESTADO INICIAL)
print (f'Lista de estados:')
print (path_states(sol))

# NÚMEROS DE ESTADOS DISTINTOS (SIN CONTAR EL ESTADO INICIAL)
print (f'Longitud lista de estados: {len(path_states(sol))-1}')

# NÚMEROS DE ESTADOS DISTINTOS (CONTANDO EL ESTADO INICIAL)
print (f'Longitud lista de estados: {len(path_states(sol)) }')

# CONJUNTO DE TODOS LOS ESTADOS EN TABLEROS
for s in path_states(sol):
    print(board8(s))

Resolver el problema EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8), (0, 1, 2, 3, 4, 5, 6, 7, 8)) con Búsqueda en Anchura
Nodo solución:  <(0, 1, 2, 3, 4, 5, 6, 7, 8)>
Coste del camino: 5
Estado solución: (0, 1, 2, 3, 4, 5, 6, 7, 8)
Estado solución (tablero):
_ 1 2
3 4 5
6 7 8

Lista de acciones: [6, 7, 4, 1, 0]
Numero de acciones en el camino: 5
Lista de estados:
[(1, 4, 2, 0, 7, 5, 3, 6, 8), (1, 4, 2, 3, 7, 5, 0, 6, 8), (1, 4, 2, 3, 7, 5, 6, 0, 8), (1, 4, 2, 3, 0, 5, 6, 7, 8), (1, 0, 2, 3, 4, 5, 6, 7, 8), (0, 1, 2, 3, 4, 5, 6, 7, 8)]
Longitud lista de estados: 5
Longitud lista de estados: 6
1 4 2
_ 7 5
3 6 8

1 4 2
3 7 5
_ 6 8

1 4 2
3 7 5
6 _ 8

1 4 2
3 _ 5
6 7 8

1 _ 2
3 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



In [38]:
# INICIALIZACIÓN DEL PROBLEMA
EJERCICIO = EightPuzzle((1, 3, 5, 8, 7, 2, 4, 6, 0))
solucionEXA = breadth_first_search(EJERCICIO)

# COSTE HASTA LLEGAR A LA SOLUCIÓN (NÚMERO DE TRANSICIONES, EN EL 8 PUZZLE)
print (f'Coste del camino: {solucionEXA.path_cost:d}')

# LISTA CON EL CONJUNTO DE ACCIONES
print (f'Lista de acciones: {path_actions(solucionEXA)}')


print (f'Lista de estados: {path_states(solucionEXA)[:4]}')

for estado in path_states(solucionEXA):
    print(board8(estado))

Coste del camino: 24
Lista de acciones: [5, 4, 3, 6, 7, 8, 5, 4, 7, 6, 3, 0, 1, 4, 3, 6, 7, 8, 5, 2, 1, 4, 3, 0]
Lista de estados: [(1, 3, 5, 8, 7, 2, 4, 6, 0), (1, 3, 5, 8, 7, 0, 4, 6, 2), (1, 3, 5, 8, 0, 7, 4, 6, 2), (1, 3, 5, 0, 8, 7, 4, 6, 2)]
1 3 5
8 7 2
4 6 _

1 3 5
8 7 _
4 6 2

1 3 5
8 _ 7
4 6 2

1 3 5
_ 8 7
4 6 2

1 3 5
4 8 7
_ 6 2

1 3 5
4 8 7
6 _ 2

1 3 5
4 8 7
6 2 _

1 3 5
4 8 _
6 2 7

1 3 5
4 _ 8
6 2 7

1 3 5
4 2 8
6 _ 7

1 3 5
4 2 8
_ 6 7

1 3 5
_ 2 8
4 6 7

_ 3 5
1 2 8
4 6 7

3 _ 5
1 2 8
4 6 7

3 2 5
1 _ 8
4 6 7

3 2 5
_ 1 8
4 6 7

3 2 5
4 1 8
_ 6 7

3 2 5
4 1 8
6 _ 7

3 2 5
4 1 8
6 7 _

3 2 5
4 1 _
6 7 8

3 2 _
4 1 5
6 7 8

3 _ 2
4 1 5
6 7 8

3 1 2
4 _ 5
6 7 8

3 1 2
_ 4 5
6 7 8

_ 1 2
3 4 5
6 7 8



In [14]:
psource (Localizaciones)

In [9]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print (g2.distancia(0,1))

55.88273580792048
119.30959564041359


In [39]:
class ProblemaViajanteComercio(Problem):
    
    def __init__(self, initial, filename='./data/grafos10_10/grafo_1.txt'):
        super().__init__(initial)
        self.grafo = Localizaciones(filename)
        self.nciudades = self.grafo.nciudades
        
    def actions(self, state):
        """ 
        Devuelve las ciudades que aún no han sido visitadas.
        """
        no_visitados = set([ciudad for ciudad in self.grafo.tablaciudades.keys() if ciudad not in state])
        if no_visitados:
            return tuple(no_visitados)
        return (state[0], )
    
    def result(self, state, action):
        """ 
        Devuelve un nuevo estado añadiendo la ciudad visitada.
        """
        state = list(state)
        action_list = list(); action_list.append(action)
        return tuple(state + action_list)
    
    def is_goal(self, state):
        """ 
        Comprueba si es el estado objetivo (se han recorrido todas las ciudades y se ha vuelto a la inicial).
        """
        return self.grafo.nciudades + 1 == len(state) and state[0] == state[-1]
    
    def action_cost(self, s, action, s1):
        """ 
        Devuelve el coste del movimiento (distancia entre las ciudades).
        """
        return self.grafo.distancia(s[-1], action)
    
    def h(self, node):
        """ 
        Función heurística (por ahora sin cambios).
        """
        return 0



def resolver(instancia_del_problema):
    problema = ProblemaViajanteComercio(instancia_del_problema, filename = "./data/grafo8cidades.txt")
    resultados = list()

    # BREADTH FIRST SEARCH
    BFS = breadth_first_search(problema)
    if BFS:
        nodos_recorridos_BFS = len(list(path_states(BFS)))  # Contamos el número de nodos recorridos
        resultados.append(("BFS", BFS.path_cost, nodos_recorridos_BFS))
    else:
        resultados.append(("BFS", "No hay", "N/A"))

    # DEPTH FIRST RECURSIVE SEARCH
    DFRS = depth_first_recursive_search(problema)
    if DFRS:
        nodos_recorridos_DFRS = len(list(path_states(DFRS)))
        resultados.append(("DFRS", DFRS.path_cost, nodos_recorridos_DFRS))
    else:
        resultados.append(("DFRS", "No hay", "N/A"))

    # DEPTH LIMITED SEARCH
    DLS = depth_limited_search(problema, 30)
    if DLS:
        nodos_recorridos_DLS = len(list(path_states(DLS)))
        resultados.append(("DLS", DLS.path_cost, nodos_recorridos_DLS))
    else:
        resultados.append(("DLS", "No hay", "N/A"))

    # Imprimimos los resultados
    print(f"Resultados de la instancia del problema: {instancia_del_problema}")
    print("| Búsqueda | Costo de la solución | Número de nodos recorridos |")
    print("|----------|----------------------|----------------------------|")
    for busqueda, coste_sol, num_nodos in resultados:
        print(f"|   {busqueda}   |    {coste_sol}    |             {num_nodos}             |")
    print("\n")

In [40]:
instancia_1 = (0, ); resolver(instancia_1)
instancia_2 = (1, ); resolver(instancia_2)
instancia_3 = (2, ); resolver(instancia_3)

Resultados de la instancia del problema: (0,)
| Búsqueda | Costo de la solución | Número de nodos recorridos |
|----------|----------------------|----------------------------|
|   BFS   |    381.6699617675482    |             9             |
|   DFRS   |    381.6699617675482    |             9             |
|   DLS   |    381.6699617675482    |             9             |


Resultados de la instancia del problema: (1,)
| Búsqueda | Costo de la solución | Número de nodos recorridos |
|----------|----------------------|----------------------------|
|   BFS   |    463.1982008134562    |             9             |
|   DFRS   |    463.1982008134562    |             9             |
|   DLS   |    463.19820081345625    |             9             |


Resultados de la instancia del problema: (2,)
| Búsqueda | Costo de la solución | Número de nodos recorridos |
|----------|----------------------|----------------------------|
|   BFS   |    526.6090019272101    |             9             |
|  

In [41]:
def resolver(instancia_del_problema):
    problema = ProblemaViajanteComercio(instancia_del_problema, filename="./data/grafos10_10/grafo_1.txt")
    resultados = list()

    # ESTRATEGIAS DE BÚSQUEDA
    estrategias = [
        ("BFS", breadth_first_search)
    ]

    # Ejecutamos cada estrategia
    for nombre, metodo in estrategias:
        sol = metodo(problema)  # Ejecutamos la búsqueda

        if sol:
            nodos_recorridos = len(list(path_states(sol)))
            resultados.append((nombre, sol.path_cost, nodos_recorridos, sol))
        else:
            resultados.append((nombre, "No hay", "N/A", None))

    # Imprimimos resultados en tabla
    print(f"\n### Resultados para la instancia {instancia_del_problema}\n")
    print("| Método | Costo de la solución | Nodos recorridos |")
    print("|--------|---------------------|------------------|")
    for metodo, costo, nodos, _ in resultados:
        print(f"| {metodo} | {costo} | {nodos} |")

    # Imprimimos detalles de cada solución encontrada
    for metodo, costo, nodos, sol in resultados:
        if sol:
            print(f"\n## Detalles de la solución con {metodo}:\n")
            print(f"🔹 Coste del camino: {sol.path_cost}")
            print(f"🔹 Estado final de la solución: {sol.state}")
            print(f"🔹 Lista de acciones: {path_actions(sol)}")
            print(f"🔹 Número de acciones en el camino: {len(path_actions(sol))}")
            print(f"🔹 Lista de estados visitados:")
            
            # Mostramos cada estado formateado
            for s in path_states(sol):
                print(s)  # Aquí podrías cambiar por una función que muestre un tablero si fuera aplicable

    print("\n" + "=" * 50 + "\n")


# Ejecutamos el código con diferentes instancias
instancias = [(0,), (1,), (2,)]
for instancia in instancias:
    resolver(instancia)



### Resultados para la instancia (0,)

| Método | Costo de la solución | Nodos recorridos |
|--------|---------------------|------------------|
| BFS | 863.0643052155478 | 11 |

## Detalles de la solución con BFS:

🔹 Coste del camino: 863.0643052155478
🔹 Estado final de la solución: (0, 1, 2, 3, 4, 5, 8, 9, 6, 7, 0)
🔹 Lista de acciones: [1, 2, 3, 4, 5, 8, 9, 6, 7, 0]
🔹 Número de acciones en el camino: 10
🔹 Lista de estados visitados:
(0,)
(0, 1)
(0, 1, 2)
(0, 1, 2, 3)
(0, 1, 2, 3, 4)
(0, 1, 2, 3, 4, 5)
(0, 1, 2, 3, 4, 5, 8)
(0, 1, 2, 3, 4, 5, 8, 9)
(0, 1, 2, 3, 4, 5, 8, 9, 6)
(0, 1, 2, 3, 4, 5, 8, 9, 6, 7)
(0, 1, 2, 3, 4, 5, 8, 9, 6, 7, 0)



### Resultados para la instancia (1,)

| Método | Costo de la solución | Nodos recorridos |
|--------|---------------------|------------------|
| BFS | 858.3771698768298 | 11 |

## Detalles de la solución con BFS:

🔹 Coste del camino: 858.3771698768298
🔹 Estado final de la solución: (1, 0, 2, 3, 4, 5, 8, 9, 6, 7, 1)
🔹 Lista de acciones: [0, 2, 3,

In [133]:
def calcular_distancia(grafo, ciudad1, ciudad2):
    return grafo.distancia(ciudad1, ciudad2)

grafo = Localizaciones("./data/grafo8cidades.txt")

# Definimos dos ciudades
ciudad_origen = 1
ciudad_destino = 3

# Calculamos la distancia entre ambas ciudades
distancia = calcular_distancia(grafo, ciudad_origen, ciudad_destino)

print(f"La distancia entre la ciudad {ciudad_origen} y la ciudad {ciudad_destino} es: {distancia}")


La distancia entre la ciudad 1 y la ciudad 3 es: 78.9222108142642
